# Trabajo práctico 2

### Obtener datasets

In [280]:
!kaggle competitions download -c tp-n2-aprendizaje-profundo-2021-by-datitos-v2

100%|██████████████████████████████████████| 1.63M/1.63M [00:00<00:00, 1.81MB/s]
100%|██████████████████████████████████████| 1.63M/1.63M [00:00<00:00, 1.81MB/s]


In [275]:
!unzip tp-n2-aprendizaje-profundo-2021-by-datitos-v2.zip

Archive:  tp-n2-aprendizaje-profundo-2021-by-datitos-v2.zip
  inflating: fifa2021_test.csv       
  inflating: fifa2021_training.csv   


### Cargar datasets

In [149]:
import pandas as pd
import numpy as np

df = pd.read_csv('fifa2021_training.csv')
df_infer = pd.read_csv('fifa2021_test.csv')

### Breve análisis exploratorio

In [123]:
df.iloc[0]

ID                            243620
Name                   Adam Hellborg
Natinality                    Sweden
Overal                            64
Potential                         73
Height                           188
Weight                            79
PreferredFoot                      R
BirthDate              July 30, 1998
Age                               22
PlayerWorkRate         Medium/Medium
WeakFoot                           3
SkillMoves                         2
Value                        1.2e+06
Wage                            1500
Club                       IK Sirius
Club_KitNumber                     2
Club_JoinedClub         Jan. 8, 2020
Club_ContractLength             2022
BallControl                       62
Dribbling                         55
Marking                           60
SlideTackle                       57
StandTackle                       60
Aggression                        71
Reactions                         58
Interceptions                     60
V

### Particionar datasets

In [122]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df, stratify=df.Position, train_size=0.9, random_state=42)

### Definir transformaciones

In [124]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer

variables_descartar = [
    'Position', # variable objetivo
    'ID',
    'Name',
    'Natinality',
    'BirthDate',
    'Value',
    'Wage',
    'Club',
    'Club_KitNumber',
    'Club_JoinedClub',
    'Club_ContractLength',
]

variables_categóricas = df.drop(columns=variables_descartar).select_dtypes(include=np.object).columns
variables_numéricas   = df.drop(columns=variables_descartar).select_dtypes(include=np.number).columns

transformador = make_column_transformer(
    (OneHotEncoder(),  variables_categóricas), # PreferredFoot, PlayerWorkRate, Sex
    (StandardScaler(), variables_numéricas),   # Overal, Potential, Height, etc.
    remainder='drop' # descarta las columnas no mencionadas en las transformaciones
)

### Entrenar transformador

Esencialmente, calculamos los **promedios** y las **desviaciones estándares** que usaremos para la estandarización.

**ESTOS VALORES SOLO DEBEN SER OBTENIDOS DEL DATASET DE ENTRENAMIENTO**.

In [125]:
transformador.fit(df_train)

ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 Index(['PreferredFoot', 'PlayerWorkRate', 'Sex'], dtype='object')),
                                ('standardscaler', StandardScaler(),
                                 Index(['Overal', 'Potential', 'Height', 'Weight', 'Age', 'WeakFoot',
       'SkillMoves', 'BallControl', 'Dribbling', 'Marking', 'SlideTackle',
       'StandTackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'ShortPass', 'LongPass', 'Acceleration',
       'Stamina', 'Strength', 'Balance', 'SprintSpeed', 'Agility', 'Jumping',
       'Heading', 'ShotPower', 'Finishing', 'LongShots', 'Curve', 'FKAcc',
       'Penalties', 'Volleys', 'GKDiving', 'GKHandling', 'GKKicking',
       'GKReflexes'],
      dtype='object'))])

### Transformar datasets

In [243]:
# el transformador se queja si falta alguna columna de df_train :/
df_infer['Position'] = None


X_train = transformador.transform(df_train)
X_valid = transformador.transform(df_valid)
X_infer = transformador.transform(df_infer)

### Transformar variable objetivo

Como la variable objetivo es del tipo string, hay que llevarla a un tipo numérico para que PyTorch pueda procesarla.

Este transformador mapea posiciones DEF, FWD, GK, MID a enteros **y viceversa** — la transformación inversa será útil para convertir las predicciones (enteros) en posiciones otra vez.

In [244]:
from sklearn.preprocessing import LabelEncoder

transformador_etiquetas = LabelEncoder()

transformador_etiquetas.fit(df_train.Position)

y_train = transformador_etiquetas.transform(df_train.Position)
y_valid = transformador_etiquetas.transform(df_valid.Position)

### Instanciar Datasets de PyTorch

El aprendizaje profundo es especialmente efectivo para imágenes y texto; para datos tabulares (como un DataFrame) el aprendizaje de máquinas clásico suele funcionar bastante bien, de ahí que PyTorch no cuente con facilidades para tratar este tipo de problemas.

In [208]:
from torch.utils.data import Dataset

class Tabular(Dataset):
    def __init__(self, X, y=None):
        self.X = X.astype(np.float32) # soluciona "Expected object of scalar type Float but got scalar type Double"
        self.y = y 

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, item):
        if self.y is None:
            return self.X[item]
        else:
            return self.X[item], self.y[item]

        
ds_train = Tabular(X_train, y_train)

Como el dataset es liviano y entra en la memoria, para validación e inferencia en vez de hacer esto

```python
ds_valid = Tabular(X_valid, y_valid)
ds_infer = Tabular(X_infer)
```

vamos a usar tensores simplemente para no complicarla.

In [203]:
ds_train[10]

(array([ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  2.2005258 ,  1.7577571 ,
         1.6827489 ,  0.69650096,  1.9552506 ,  2.9999895 , -0.49693993,
         1.5163243 ,  1.0263261 ,  0.5255314 , -1.4219726 , -0.93868953,
         0.10657337,  2.1206129 ,  0.13522774,  1.5071335 ,  2.1563685 ,
         0.6801946 ,  1.1708272 ,  1.3763508 , -0.25280562,  0.7491539 ,
         1.286718  , -2.0082333 , -0.4727765 , -0.531109  , -0.06149001,
         1.7349557 ,  2.030825  ,  2.0725315 ,  1.5956575 ,  0.90770805,
         0.6198386 ,  1.1499894 ,  1.857724  , -0.42461264, -0.31164196,
        -0.19319764, -0.09562566], dtype=float32),
 1)

### Instanciar DataLoaders de PyTorch

In [187]:
from torch.utils.data import DataLoader

dl_train = DataLoader(ds_train, batch_size=32, shuffle=True)

Una de las cosas que `DataLoader` hace es convertir arreglos de NumPy en tensores de PyTorch.

Como no vamos a hacer esto

```python
# sin shuffle porque validación e inferencia no requieren barajar sus elementos 
dl_valid = DataLoader(ds_valid, batch_size=32)
dl_infer = DataLoader(ds_infer, batch_size=32)
```

vamos a definir tensores a mano.

In [245]:
X_valid = torch.tensor(X_valid).float()
X_infer = torch.tensor(X_infer).float()

y_valid = torch.tensor(y_valid)

### Instanciar modelo

In [260]:
import torch
import torch.nn as nn

In [261]:
IN  = X_trans.shape[1]
OUT = len(transformador_etiquetas.classes_)

modelo = nn.Sequential(
    nn.Linear(IN,  8),
    nn.Linear( 8, 64), nn.ReLU(),
    nn.Linear(64, 32), nn.ReLU(),
    nn.Linear(32, OUT)
)

In [262]:
criterio = nn.CrossEntropyLoss()
optimizador = torch.optim.Adam(modelo.parameters(), lr=0.0001)

### Entrenar modelo

In [263]:
from sklearn.metrics import balanced_accuracy_score

ÉPOCAS = 10

for época in range(ÉPOCAS):
    # activa capas Dropout, BatchNorm si las hubiese
    modelo.train()

    pérdidas_train = []
    
    for X_lote, y_lote in dl_train:
        optimizador.zero_grad()

        predicciones = modelo(X_lote)
        pérdida = criterio(predicciones, y_lote)

        pérdida.backward()
        optimizador.step()
        
        pérdidas_train.append(pérdida.item())
    
    # desactiva capas Dropout, BatchNorm si las hubiese
    modelo.eval()
    
    with torch.no_grad():
        predicciones = modelo(X_valid)
        pérdida = criterio(predicciones, y_valid)
        
        y_pred = predicciones.argmax(dim=1) # selecciona la clase con mayor probabilidad
        
        efectividad = balanced_accuracy_score(y_valid, y_pred)
    
    
    print(f'{época:3d}  |  Train loss: {np.mean(pérdidas_train):.3f}    Valid loss: {pérdida:.3f}    Valid accuracy: {efectividad:.2f}')

  0  |  Train loss: 1.171    Valid loss: 0.823    Valid accuracy: 0.67
  1  |  Train loss: 0.616    Valid loss: 0.472    Valid accuracy: 0.82
  2  |  Train loss: 0.396    Valid loss: 0.355    Valid accuracy: 0.87
  3  |  Train loss: 0.325    Valid loss: 0.321    Valid accuracy: 0.88
  4  |  Train loss: 0.300    Valid loss: 0.304    Valid accuracy: 0.88
  5  |  Train loss: 0.288    Valid loss: 0.295    Valid accuracy: 0.88
  6  |  Train loss: 0.279    Valid loss: 0.288    Valid accuracy: 0.88
  7  |  Train loss: 0.274    Valid loss: 0.282    Valid accuracy: 0.88
  8  |  Train loss: 0.269    Valid loss: 0.277    Valid accuracy: 0.89
  9  |  Train loss: 0.265    Valid loss: 0.276    Valid accuracy: 0.89


### Inferir datos de prueba

In [265]:
with torch.no_grad():
    y_infer = modelo(X_infer).argmax(dim=1)

df_infer['Position'] = transformador_etiquetas.inverse_transform(y_infer)

(
    df_infer[['ID', 'Position']]
    .rename(columns={'ID':'Id', 'Position':'Category'})
    .to_csv('submit.csv', index=False)
)

### Subir predicciones

In [281]:
!kaggle competitions submit -c tp-n2-aprendizaje-profundo-2021-by-datitos-v2 -f submit.csv -m "Brasil, decime qué se siente"

100%|██████████████████████████████████████| 62.8k/62.8k [00:02<00:00, 26.4kB/s]
Successfully submitted to T.P. N°2 - Aprendizaje Profundo 2021 by Datitos